In [5]:
pip install pygame

Note: you may need to restart the kernel to use updated packages.


In [41]:
import random
import numpy as np

# Parameter algoritma genetika
population_size = 12
num_generations = 12
mutation_rate = 0.07
crossover_rate = 0.9

# Fungsi untuk menginisialisasi populasi (set durasi lampu hijau secara acak)
def initialize_population():
    return [np.random.randint(3, 7, size=4).tolist() for _ in range(population_size)]  # 4 arah

# Fungsi kebugaran: menghitung total waktu tunggu berdasarkan durasi lampu hijau
def fitness_function(chromosome, traffic_data):
    total_wait_time = 0
    for i, duration in enumerate(chromosome):
        num_vehicles = traffic_data[i]
        total_wait_time += num_vehicles * duration  # Semakin banyak kendaraan, semakin lama waktu tunggu
    return total_wait_time

# Seleksi kromosom terbaik (Roulette Wheel Selection)
def selection(population, fitness_scores):
    total_fitness = sum(fitness_scores)
    selection_probs = [fitness / total_fitness for fitness in fitness_scores]
    return population[np.random.choice(len(population), p=selection_probs)]

# Penyilangan (Crossover)
def crossover(parent1, parent2):
    if random.random() < crossover_rate:
        crossover_point = random.randint(1, len(parent1) - 1)
        child1 = parent1[:crossover_point] + parent2[crossover_point:]
        child2 = parent2[:crossover_point] + parent1[crossover_point:]
        return child1, child2
    return parent1, parent2

# Mutasi (Mutation)
def mutate(chromosome):
    if random.random() < mutation_rate:
        mutate_index = random.randint(0, len(chromosome) - 1)
        chromosome[mutate_index] = random.randint(3, 7)  # Mutasi durasi lampu hijau antara 3-10 detik
    return chromosome

# Algoritma genetika untuk menemukan durasi optimal
def genetic_algorithm(traffic_data):
    population = initialize_population()

    for generation in range(num_generations):
        fitness_scores = [fitness_function(chromosome, traffic_data) for chromosome in population]

        new_population = []
        for _ in range(population_size // 2):
            parent1 = selection(population, fitness_scores)
            parent2 = selection(population, fitness_scores)
            child1, child2 = crossover(parent1, parent2)
            new_population.append(mutate(child1))
            new_population.append(mutate(child2))

        population = new_population

        # Cari kromosom dengan kebugaran terbaik
        best_chromosome = min(population, key=lambda chromo: fitness_function(chromo, traffic_data))
        best_fitness = fitness_function(best_chromosome, traffic_data)

        print(f"Generation {generation + 1}: Best Fitness = {best_fitness}, Best Chromosome = {best_chromosome}")

    return best_chromosome


In [43]:
import pygame
import random
import time

# Inisialisasi Pygame
pygame.init()

# Ukuran layar
screen_width = 600
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("Traffic Simulation")

# Warna
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
BLACK = (0, 0, 0)
YELLOW = (255, 203, 46)

# Kecepatan kendaraan
vehicle_speed = 2

# Lampu Lalu Lintas (Awal semua merah kecuali East)
traffic_lights = {'North': YELLOW, 'South': RED, 'East': RED, 'West': RED}
green_light_duration = 3  # Durasi lampu hijau (5 detik)
light_timer = 0  # Timer untuk menghitung waktu lampu hijau
current_green_direction = None  # Arah awal dengan lampu hijau
current_index = 0 # Indeks untuk melacak giliran arah yang akan mendapat lampu hijau
light_sequence = ['North', 'South', 'East', 'West']
optimal_durations = [2, 0, 0, 0]

# Kendaraan class
class Vehicle:
    def __init__(self, x, y, direction):
        self.x = x
        self.y = y
        self.direction = direction
        self.size = 20
        self.color = random.choice([RED, BLUE])
        self.has_crossed_intersection = False  # Kendaraan belum melewati persimpangan
        self.queue = True
        self.turn_decided = False
        self.turn = None
        self.decided_turn = None

    def decide_turn(self):
        # Memutuskan apakah kendaraan akan lurus, belok kiri, atau belok kanan
        if not self.turn_decided:  # Hanya putuskan sekali
            turn_prob = random.random()
            if turn_prob < 0.3:
                self.turn = 'left'
            elif turn_prob < 0.6:
                self.turn = 'right'
            else:
                self.turn = None  # Tetap lurus
            self.turn_decided = True  # Sudah memutuskan belok atau tidak


    def move(self, vehicles):
        # Tentukan batas garis persimpangan untuk setiap arah
        if self.direction == 'North' and self.y < screen_height // 2 + 50:
            self.has_crossed_intersection = True
        elif self.direction == 'South' and self.y > screen_height // 2 - 50:
            self.has_crossed_intersection = True
        elif self.direction == 'East' and self.x > screen_width // 2 - 50:
            self.has_crossed_intersection = True
        elif self.direction == 'West' and self.x < screen_width // 2 + 50:
            self.has_crossed_intersection = True

        if self.direction == 'North' and self.y < screen_height // 2 + 55:
            self.queue = False
        elif self.direction == 'South' and self.y > screen_height // 2 - 75:
            self.queue = False
        elif self.direction == 'East' and self.x > screen_width // 2 - 75:
            self.queue = False
        elif self.direction == 'West' and self.x < screen_width // 2 + 55:
            self.queue = False

        # Kendaraan berhenti jika lampu merah dan belum melewati persimpangan
        if traffic_lights[self.direction] == GREEN or \
        (traffic_lights[self.direction] == YELLOW and self.has_crossed_intersection) or \
        self.has_crossed_intersection or self.queue:
            # Cek jika ada kendaraan di depan (mencegah tabrakan)
            if not self.is_collision(vehicles):
                # Tentukan apakah akan belok atau tidak setelah melewati persimpangan
                if self.has_crossed_intersection and self.turn is None:
                    self.decide_turn()
                    
                # Arahkan kendaraan jika belok
                if self.turn == 'left':
                    if self.direction == 'North' and self.y == screen_height // 2 + 20:
                        self.turn_left()
                    elif self.direction == 'South' and self.y == screen_height // 2 - 20:
                        self.turn_left()
                    elif self.direction == 'East' and self.x == screen_width // 2 - 20:
                        self.turn_left()
                    elif self.direction == 'West' and self.x == screen_width // 2 + 20:
                        self.turn_left()
                    else:
                        self.go_straight()
                        
                elif self.turn == 'right':
                    if self.direction == 'North' and self.y == screen_height // 2 - 20:
                        self.turn_right()
                    elif self.direction == 'South' and self.y == screen_height // 2 + 20:
                        self.turn_right()
                    elif self.direction == 'East' and self.x == screen_width // 2 + 20:
                        self.turn_right()
                    elif self.direction == 'West' and self.x == screen_width // 2 - 20:
                        self.turn_right()
                    else:
                        self.go_straight()
                else:
                    self.go_straight()        

    def go_straight(self):
        if self.direction == 'North':
            self.y -= vehicle_speed
        elif self.direction == 'South':
            self.y += vehicle_speed
        elif self.direction == 'East':
            self.x += vehicle_speed
        elif self.direction == 'West':
            self.x -= vehicle_speed

    def turn_left(self):
        # Belok kiri
        if self.direction == 'North':
            self.x -= vehicle_speed  # Belok ke barat

        elif self.direction == 'South':
            self.x += vehicle_speed  # Belok ke timur

        elif self.direction == 'East':
            self.y -= vehicle_speed  # Belok ke utara

        elif self.direction == 'West':
            self.y += vehicle_speed  # Belok ke selatan


    def turn_right(self):
        # Belok kanan
        if self.direction == 'North':
            self.x += vehicle_speed  # Belok ke timur

        elif self.direction == 'South':
            self.x -= vehicle_speed  # Belok ke barat

        elif self.direction == 'East':
            self.y += vehicle_speed  # Belok ke selatan

        elif self.direction == 'West':
            self.y -= vehicle_speed  # Belok ke utara


    def is_collision(self, vehicles):
        # Mencegah kendaraan terlalu dekat dengan kendaraan lain di depannya
        for vehicle in vehicles:
            if vehicle != self:  # Jangan cek tabrakan dengan diri sendiri
                if self.direction == 'North' and self.x == vehicle.x and 0 < self.y - vehicle.y < self.size + 10:
                    return True
                elif self.direction == 'South' and self.x == vehicle.x and 0 < vehicle.y - self.y < self.size + 10:
                    return True
                elif self.direction == 'East' and self.y == vehicle.y and 0 < vehicle.x - self.x < self.size + 10:
                    return True
                elif self.direction == 'West' and self.y == vehicle.y and 0 < self.x - vehicle.x < self.size + 10:
                    return True
        return False

    def draw(self, screen):
        pygame.draw.rect(screen, self.color, (self.x, self.y, self.size, self.size))

# Fungsi untuk menggambar persimpangan dan lampu lalu lintas
def draw_intersection():
    pygame.draw.line(screen, BLACK, (screen_width / 2 - 50, 0), (screen_width / 2 - 50, screen_height), 5)
    pygame.draw.line(screen, BLACK, (screen_width / 2 + 50, 0), (screen_width / 2 + 50, screen_height), 5)
    pygame.draw.line(screen, BLACK, (0, screen_height / 2 - 50), (screen_width, screen_height / 2 - 50), 5)
    pygame.draw.line(screen, BLACK, (0, screen_height / 2 + 50), (screen_width, screen_height / 2 + 50), 5)

# Fungsi untuk menggambar lampu lalu lintas berdasarkan statusnya
def draw_traffic_lights():
    # Tampilkan lampu lalu lintas di setiap arah
    pygame.draw.circle(screen, traffic_lights['North'], (screen_width // 2 - 75, screen_height // 2 + 75), 20)  # North
    pygame.draw.circle(screen, traffic_lights['East'], (screen_width // 2 - 75, screen_height // 2 - 75), 20)   # East
    pygame.draw.circle(screen, traffic_lights['South'], (screen_width // 2 + 75, screen_height // 2 - 75), 20)  # South
    pygame.draw.circle(screen, traffic_lights['West'], (screen_width // 2 + 75, screen_height // 2 + 75), 20)   # West

# Fungsi untuk mengganti lampu lalu lintas berdasarkan jumlah kendaraan

def change_traffic_lights(vehicles):
    global light_timer, current_green_direction, green_light_duration, current_index, optimal_durations

    # Memastikan lampu kuning menyala 1 detik sebelum waktu hijau habis
    if light_timer >= (green_light_duration - 1) * 60 and light_timer < green_light_duration * 60:
        traffic_lights[current_green_direction] = YELLOW
    
    elif light_timer >= green_light_duration * 60:
        for direction in traffic_lights:
            traffic_lights[direction] = RED

        # Set urutan berikutnya dan atur lampu hijau dengan durasi optimal
        current_index = (current_index + 1) % len(light_sequence)
        current_green_direction = light_sequence[current_index]
        
        # Terapkan durasi optimal untuk arah saat ini
        green_light_duration = optimal_durations[current_index]
        traffic_lights[current_green_direction] = GREEN
        
        # Reset timer
        light_timer = 0

        # Jika semua arah sudah dilalui, hitung ulang durasi optimal dengan algoritma genetika
        if current_index == 0:
            direction_counts = [0, 0, 0, 0]  # North, South, East, West
            for vehicle in vehicles:
                idx = ['North', 'South', 'East', 'West'].index(vehicle.direction)
                direction_counts[idx] += 1
                if vehicle.has_crossed_intersection:
                    direction_counts[idx] -= 1
            
            # Hitung durasi optimal dengan algoritma genetika
            optimal_durations = genetic_algorithm(direction_counts)
            print(f"Optimal Green Light Durations: {optimal_durations}")
    
    # Tetapkan lampu hijau saat timer berjalan
    else:
        traffic_lights[current_green_direction] = GREEN
    
    # Naikkan timer setiap frame
    light_timer += 1

# Membuat kendaraan
def generate_vehicle(direction):
    if direction == 'North':
        return Vehicle(screen_width // 2 - 20, screen_height, 'North')
    elif direction == 'South':
        return Vehicle(screen_width // 2 + 20, 0, 'South')
    elif direction == 'East':
        return Vehicle(0, screen_height // 2 - 20, 'East')
    elif direction == 'West':
        return Vehicle(screen_width, screen_height // 2 + 20, 'West')

# Main loop
def traffic_simulation():
    clock = pygame.time.Clock()
    running = True
    vehicles = []
    
    # Tambahkan kendaraan dari arah yang berbeda
    vehicle_directions = ['North', 'South', 'East', 'West']
    
    while running:
        screen.fill(WHITE)
        draw_intersection()
        draw_traffic_lights()
        
        # Buat kendaraan secara acak (mengurangi frekuensi)
        if random.randint(1, 50) == 1:  # Frekuensi muncul lebih jarang
            direction = random.choice(vehicle_directions)
            vehicles.append(generate_vehicle(direction))
        
        # Gerakkan kendaraan dan gambar di layar
        for vehicle in vehicles:
            vehicle.move(vehicles)
            vehicle.draw(screen)

        # Ubah lampu lalu lintas berdasarkan jumlah kendaraan
        change_traffic_lights(vehicles)
        
        # Periksa event Pygame
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
        
        pygame.display.update()
        clock.tick(60)  # FPS 60
        
    pygame.quit()

# Jalankan simulasi lalu lintas
traffic_simulation()


Generation 1: Best Fitness = 27, Best Chromosome = [3, 5, 4, 4]
Generation 2: Best Fitness = 29, Best Chromosome = [3, 6, 4, 6]
Generation 3: Best Fitness = 31, Best Chromosome = [4, 6, 3, 5]
Generation 4: Best Fitness = 31, Best Chromosome = [4, 6, 3, 5]
Generation 5: Best Fitness = 30, Best Chromosome = [4, 6, 3, 4]
Generation 6: Best Fitness = 30, Best Chromosome = [4, 5, 3, 4]
Generation 7: Best Fitness = 30, Best Chromosome = [4, 6, 3, 4]
Generation 8: Best Fitness = 35, Best Chromosome = [5, 6, 3, 4]
Generation 9: Best Fitness = 35, Best Chromosome = [5, 6, 3, 4]
Generation 10: Best Fitness = 35, Best Chromosome = [5, 6, 3, 4]
Generation 11: Best Fitness = 35, Best Chromosome = [5, 6, 3, 4]
Generation 12: Best Fitness = 35, Best Chromosome = [5, 6, 3, 4]
Optimal Green Light Durations: [5, 6, 3, 4]
Generation 1: Best Fitness = 47, Best Chromosome = [4, 5, 6, 3]
Generation 2: Best Fitness = 44, Best Chromosome = [4, 5, 3, 3]
Generation 3: Best Fitness = 45, Best Chromosome = [4, 3,